In [ ]:
import geopandas as gpd
import osmnx as ox
import numpy as np
import networkx as nx
import pandas as pd
import fiona
import folium
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from IPython.display import Image

%matplotlib inline
ox.__version__

In [ ]:
# Get a driving network
city_name = "Hamilton, NZ"
G = ox.graph_from_place(city_name, network_type="drive")
fig, ax = ox.plot_graph(G)

In [ ]:
# Stats about the road network
G_proj = ox.project_graph(G)
nodes_proj = ox.graph_to_gdfs(G_proj, edges=False)
graph_area_km = nodes_proj.unary_union.convex_hull.area/10**6

print(ox.basic_stats(G_proj, area=graph_area_km, clean_int_tol=15))

In [ ]:
# get rail networks from various cities
cities = [
    "Paris, France",
    "London, England",
    "Oslo, Norway",
    "Stockholm, Sweden",
]
city_single = ["Stockholm, Sweden"]
custom_filter = '["railway"]'
amenities = ["rail", "subway", "light_rail", "tram", "monorail"]
city_center = ox.geocode(city_single[0])
m = folium.Map(location=[city_center[0], city_center[1]], zoom_start=5)

# Create an empty graph to combine all city graphs
combined_graph = nx.MultiDiGraph()

# Iterate over each city and get the rail network
for amenity in amenities:
    try:
        G = ox.graph_from_place(
            city_single[0],
            retain_all=False,
            truncate_by_edge=False,
            simplify=True,
            custom_filter=f'["railway"~"{amenity}"]["service"!~"yard"]',
        )
        for u, v, k, data in G.edges(keys=True, data=True):
            data["rail_type"] = amenity
        combined_graph = nx.compose(combined_graph, G)
    except Exception as e:
            print(f"Error processing {city_single[0]} with amenity {amenity}: {e}")
    
gdf_nodes, gdf_edges = ox.graph_to_gdfs(combined_graph)
print(gdf_edges.head())

gdf_edges.to_file(
    "./data/rail_lines.gpkg",
    layer="rail_lines_stockholm",
    driver="GPKG",
)
#ox.save_graph_geopackage(G, filepath="./data/paris_rail_lines.gpkg")

In [ ]:
m = folium.Map(location=[city_center[0], city_center[1]], zoom_start=5)
print(combined_graph)

In [ ]:
color_mapping = {
    "rail": "blue",
    "subway": "red",
    "light_rail": "green",
    "tram": "orange",
    "monorail": "purple"
}

for _, row in gdf_edges.iterrows():
        name = row.get("name", "N/A")
        service = row.get("service", "N/A")
        rail_type = row.get("rail_type", "N/A")
        color = color_mapping.get(rail_type, "blue")
        popup_text = f"Name: {name}<br>Service: {service}<br>Rail Type: {rail_type}"

        folium.PolyLine(
            locations=[(point[1], point[0]) for point in row["geometry"].coords],
            color=color,
            weight=2,
            popup=folium.Popup(popup_text, max_width=300)
        ).add_to(m)

m